# **SEMANTIC SEGMENTATION**

**Devices**

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2024-04-07 17:50:59.145301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 17:50:59.145438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 17:50:59.285584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2958070907211393889
xla_global_id: -1
]


**Imports**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm
from glob import glob
import cv2
import albumentations as A
from skimage.transform import resize
from sklearn.model_selection import train_test_split

import tensorflow as tf
from skimage.transform import resize
from tensorflow.keras.saving import load_model
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Lambda
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

In [4]:
def seed_everything(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_everything()

## **DATASET**

In [5]:
H, W = 192, 192
num_patches = 21*32
shape = (H, W, 3)
num_classes = 23
lr = 1e-4
batch_size = 1
epochs = 30

In [6]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [9]:
image_path = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images'
label_path = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic'

def create_dataframe(path):
    name = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            name.append(filename.split('.')[0])

    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df_images = create_dataframe(image_path)
df_masks = create_dataframe(label_path)
print('Total Images: ', len(df_images))

Total Images:  400


In [10]:
X_trainval, X_test = train_test_split(df_images['id'], test_size=0.1)
X_train, X_val = train_test_split(X_trainval, test_size=0.2)

print(f"Train Size : {len(X_train)} images")
print(f"Val Size   :  {len(X_val)} images")
print(f"Test Size  :  {len(X_test)} images")

y_train = X_train
y_test = X_test
y_val = X_val

img_train = [os.path.join(image_path, f"{name}.jpg") for name in X_train]
mask_train = [os.path.join(label_path, f"{name}.png") for name in y_train]
img_val = [os.path.join(image_path, f"{name}.jpg") for name in X_val]
mask_val = [os.path.join(label_path, f"{name}.png") for name in y_val]
img_test = [os.path.join(image_path, f"{name}.jpg") for name in X_test]
mask_test = [os.path.join(label_path, f"{name}.png") for name in y_test]

Train Size : 288 images
Val Size   :  72 images
Test Size  :  40 images


In [11]:
def read_images(image_path, mask_path):
    '''Reads an image and its corresponding mask from their file paths.'''
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)

    return image, mask


def pad_image(image, mask):
    '''Pads the image and mask with constant values to ensure consistent dimensions.'''
    image = tf.pad(image, [[16, 16], [72, 72], [0, 0]], constant_values=-1)
    mask = tf.pad(mask, [[16, 16], [72, 72], [0, 0]], constant_values=-1)

    return image, mask


def split_images(image, mask):
    '''Splits the image and mask into patches of the specified size.'''
    image_patches = tf.image.extract_patches(image[tf.newaxis, ...], sizes=[1, 192, 192, 1], strides=[1, 192, 192, 1], rates=[1, 1, 1, 1], padding='VALID')
    mask_patches = tf.image.extract_patches(mask[tf.newaxis, ...], sizes=[1, 192, 192, 1], strides=[1, 192, 192, 1], rates=[1, 1, 1, 1], padding='VALID')
    image_patches = tf.reshape(image_patches, (-1, 192, 192, 3))
    mask_patches = tf.reshape(mask_patches, (-1, 192, 192, 1))

    return image_patches, mask_patches


def save_images(image_patches, mask_patches, image_dir, mask_dir, name):
    '''Saves all the image and mask patches to specified directory.'''
    ROWS = 21
    COLS = 32

    for i, (image_patch, mask_patch) in enumerate(zip(image_patches, mask_patches)):

        image_patch = tf.cast(image_patch, tf.uint8)
        mask_patch = tf.cast(mask_patch, tf.uint8)

        image_filename = os.path.join(image_dir, f"{name:3}_{i//COLS}_{i%COLS}.jpg")
        mask_filename = os.path.join(mask_dir, f"{name:3}_{i//COLS}_{i%COLS}.png")

        tf.io.write_file(image_filename, tf.image.encode_png(image_patch))
        tf.io.write_file(mask_filename, tf.image.encode_png(mask_patch))


def create_new_dataset(output_dir, img_data, mask_data):
    '''Function encapsulates the entire process of reading in the original images, padding, splitting and saving the patches.'''
    image_dir, mask_dir = os.path.join(output_dir, 'images'), os.path.join(output_dir, 'masks')
    create_dir(image_dir)
    create_dir(mask_dir)

    for i, (image_path, mask_path) in tqdm(enumerate(zip(img_data, mask_data)), total=len(img_data)):
        image, mask = read_images(image_path, mask_path)
        image, mask = pad_image(image, mask)
        image_patches, mask_patches = split_images(image, mask)
        save_images(image_patches, mask_patches, image_dir, mask_dir, i)

In [13]:
%%time
create_new_dataset('/kaggle/working/dataset/train', img_train, mask_train)

100%|██████████| 288/288 [1:19:06<00:00, 16.48s/it]

CPU times: user 1h 31min 13s, sys: 1min 51s, total: 1h 33min 5s
Wall time: 1h 19min 6s


In [14]:
%%time
create_new_dataset('/kaggle/working/dataset/val', img_val, mask_val)

100%|██████████| 72/72 [20:05<00:00, 16.74s/it]

CPU times: user 23min 4s, sys: 34.2 s, total: 23min 38s
Wall time: 20min 5s


In [15]:
%%time
create_new_dataset('/kaggle/working/dataset/test', img_test, mask_test)

100%|██████████| 40/40 [11:17<00:00, 16.93s/it]

CPU times: user 12min 54s, sys: 19.5 s, total: 13min 13s
Wall time: 11min 17s


In [19]:
import shutil
shutil.make_archive('dataset', 'zip', '/kaggle/working/dataset')

OSError: [Errno 28] No space left on device

In [20]:
! pip install kaggle

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
! kaggle datasets download -d mrandes/dataset-name -p /path/to/download

In [37]:
import shutil
shutil.make_archive('/kaggle/input/', 'zip', '/kaggle/working/dataset/')

OSError: [Errno 30] Read-only file system: '/kaggle/input/.zip'

## **MODEL EXPERIMENTATION**

In [ ]:
epochs = 1

### UNET

In [ ]:
from skimage import io

image = io.imread('https://github.com/qubvel/segmentation_models/raw/master/images/unet.png')
plt.imshow(image)

In [ ]:
def contracting_path(x, filters):
    conv = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    conv = Conv2D(filters, (3, 3), activation='relu', padding='same')(conv)
    pool = MaxPooling2D((2, 2))(conv)
    return conv, pool

def expanding_path(x, skip_connections, filters):
    up = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(x)
    cat = concatenate([up, skip_connections])
    conv = Conv2D(filters, (3, 3), activation='relu', padding='same')(cat)
    conv = Conv2D(filters, (3, 3), activation='relu', padding='same')(conv)
    return conv

def build_unet(input_shape=shape, num_classes=num_classes):

    inputs = Input(input_shape)

    c1, p1 = contracting_path(inputs, 16)
    c2, p2 = contracting_path(p1, 32)
    c3, p3 = contracting_path(p2, 64)
    c4, p4 = contracting_path(p3, 128)

    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u4 = expanding_path(c5, c4, 128)
    u3 = expanding_path(u4, c3, 64)
    u2 = expanding_path(u3, c2, 32)
    u1 = expanding_path(u2, c1, 16)

    outputs = Conv2D(1, (1, 1), padding='same', activation='softmax')(u1)

    model = Model (inputs=inputs, outputs=outputs)

    return model


In [ ]:
unet = build_unet()

unet.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr),
              metrics=['accuracy']
              )
unet.summary()